In [1]:
import uuid
import random
import gym
import numpy as np

class Node:
    def __init__(self, state, action, action_space, reward, terminal):
        self.identifier = str(uuid.uuid1())
        self.parent_identifier = None
        self.children_identifiers = []
        self.untried_actions = list(range(action_space))
        self.state = state
        self.total_simulation_reward = 0
        self.num_visits = 0
        self.performance = 0
        self.action = action
        self.reward = reward
        self.terminal = terminal
        self.q_values = np.zeros(action_space)  # Q-values for actions
        self.action_visits = np.zeros(action_space)  # Count visits for each action

    def __str__(self):
        return "{}: (action={}, visits={}, reward={:d}, ratio={:0.4f})".format(
                                                  self.state,
                                                  self.action,
                                                  self.num_visits,
                                                  int(self.total_simulation_reward),
                                                  self.performance)

    def untried_action(self):
        action = random.choice(self.untried_actions)
        self.untried_actions.remove(action)
        return action

class Tree:
    def __init__(self):
        self.nodes = {}
        self.root = None

    def is_expandable(self, node):
        return len(node.untried_actions) > 0 and not node.terminal

    def iter(self, identifier, depth, last_node_flags):
        if identifier is None:
            node = self.root
        else:
            node = self.nodes[identifier]

        if depth == 0:
            yield "", node
        else:
            yield vertical_lines(last_node_flags) + horizontal_line(last_node_flags), node

        children = [self.nodes[identifier] for identifier in node.children_identifiers]
        last_index = len(children) - 1

        depth += 1
        for index, child in enumerate(children):
            last_node_flags.append(index == last_index)
            for edge, node in self.iter(child.identifier, depth, last_node_flags):
                yield edge, node
            last_node_flags.pop()

    def add_node(self, node, parent=None):
        self.nodes.update({node.identifier: node})

        if parent is None:
            self.root = node
        else:
            self.nodes[parent.identifier].children_identifiers.append(node.identifier)
            self.nodes[node.identifier].parent_identifier = parent.identifier

    def children(self, node):
        return [self.nodes[identifier] for identifier in node.children_identifiers]

    def parent(self, node):
        parent_identifier = self.nodes[node.identifier].parent_identifier
        return self.nodes.get(parent_identifier, None)

    def show(self):
        lines = ""
        for edge, node in self.iter(identifier=None, depth=0, last_node_flags=[]):
            lines += "{}{}\n".format(edge, node)
        print(lines)

def vertical_lines(last_node_flags):
    vertical_lines = []
    vertical_line = '\u2502'
    for last_node_flag in last_node_flags[0:-1]:
        if not last_node_flag:
            vertical_lines.append(vertical_line + ' ' * 3)
        else:
            vertical_lines.append(' ' * 4)
    return ''.join(vertical_lines)

def horizontal_line(last_node_flags):
    horizontal_line = '\u251c\u2500\u2500 '
    horizontal_line_end = '\u2514\u2500\u2500 '
    return horizontal_line_end if last_node_flags[-1] else horizontal_line

class MonteCarloTreeSearch:
    def __init__(self, env, tree, max_depth=10, c_param=1.4):
        self.env = env
        self.tree = tree
        self.action_space = self.env.action_space.n
        self.max_depth = max_depth
        self.c_param = c_param
        state = self.env.reset()
        self.tree.add_node(Node(state=state, action=None, action_space=self.action_space, reward=0, terminal=False))

    def expand(self, node):
        action = node.untried_action()
        state, reward, done, _, _ = self.env.step(action)
        new_node = Node(state=state, action=action, action_space=self.action_space, reward=reward, terminal=done)
        self.tree.add_node(new_node, node)
        return new_node

    def default_policy(self, node, depth):
        total_reward = node.reward
        current_depth = depth

        while not node.terminal and current_depth < self.max_depth:
            action = random.randint(0, self.action_space - 1)  # Random action
            state, reward, done, _, _ = self.env.step(action)
            total_reward += reward
            current_depth += 1

            if done:
                break

        return total_reward

    def tree_policy(self):
        node = self.tree.root
        depth = 0

        while not node.terminal and depth < self.max_depth:
            if self.tree.is_expandable(node):
                return self.expand(node)
            else:
                node = self.best_child(node)
                state, reward, done, _, _ = self.env.step(node.action)
                assert node.state == state
            depth += 1

        return node

    def best_child(self, node):
        children = self.tree.children(node)
        total_visits = np.sum([child.num_visits for child in children])
        best_value = -np.inf
        best_node = None

        for child in children:
            exploitation = child.performance
            exploration = self.c_param * np.sqrt(np.log(total_visits) / (child.num_visits + 1))
            uct_value = exploitation + exploration

            if uct_value > best_value:
                best_value = uct_value
                best_node = child

        return best_node

    def backward(self, node, value):
        while node:
            node.num_visits += 1
            node.total_simulation_reward += value
            node.performance = node.total_simulation_reward / node.num_visits

            if node.action is not None:
                node.action_visits[node.action] += 1
                node.q_values[node.action] += (value - node.q_values[node.action]) / node.action_visits[node.action]

            node = self.tree.parent(node)

    def forward(self):
        self._forward(self.tree.root)

    def _forward(self, node):
        best_child = max(self.tree.children(node), key=lambda n: np.max(n.q_values), default=None)

        print("****** {} ******".format(best_child.state))

        for child in self.tree.children(best_child):
            print("{}: {:0.4f}".format(child.state, np.max(child.q_values)))

        if best_child and len(self.tree.children(best_child)) > 0:
            self._forward(best_child)

    def render_policy(self):
        node = self.tree.root
        path = []
        directions = {0: 'Left', 1: 'Down', 2: 'Right', 3: 'Up'}
        print("Rendering final policy...\n")

        # Close the current environment if open and reset with rendering enabled
        self.env.close()
        self.env = gym.make('FrozenLake-v1', is_slippery=False, render_mode='human')
        self.env.reset()

        while node and not node.terminal:
            self.env.render()
            path.append(node.state)
            children = self.tree.children(node)

            if not children:
                print("No more actions available.")
                break

            best_child = max(children, key=lambda n: np.max(n.q_values), default=None)
            if best_child is not None and best_child.action is not None:
                print(f"Action: {directions.get(best_child.action, 'Unknown')} -> State: {best_child.state}")
                self.env.step(best_child.action)  # Take the action in the environment
                node = best_child
            else:
                print("No best child found.")
                break

        # Final rendering after reaching the goal
        self.env.render()
        self.env.close()

        path2 = [item[0] if isinstance(item, tuple) else item for item in path]
        print(f"\nFinal policy path (states): {path2}")

        return path2

def main():
    env = gym.make('FrozenLake-v1', is_slippery=False)
    tree = Tree()
    monteCarloTreeSearch = MonteCarloTreeSearch(env=env, tree=tree)
    steps = 100000

    for _ in range(steps):
        env.reset()
        node = monteCarloTreeSearch.tree_policy()
        reward = monteCarloTreeSearch.default_policy(node, depth=0)
        monteCarloTreeSearch.backward(node, reward)

    monteCarloTreeSearch.tree.show()
    print("Best child choices:")
    monteCarloTreeSearch.forward()
    monteCarloTreeSearch.render_policy()

if __name__ == "__main__":
    main()


(0, {'prob': 1}): (action=None, visits=100000, reward=97646, ratio=0.9765)
├── 4: (action=1, visits=99317, reward=97642, ratio=0.9831)
│   ├── 5: (action=2, visits=57, reward=0, ratio=0.0000)
│   ├── 8: (action=1, visits=99135, reward=97640, ratio=0.9849)
│   │   ├── 4: (action=3, visits=22, reward=0, ratio=0.0000)
│   │   │   ├── 5: (action=2, visits=6, reward=0, ratio=0.0000)
│   │   │   ├── 8: (action=1, visits=5, reward=0, ratio=0.0000)
│   │   │   │   ├── 8: (action=0, visits=1, reward=0, ratio=0.0000)
│   │   │   │   ├── 9: (action=2, visits=1, reward=0, ratio=0.0000)
│   │   │   │   ├── 4: (action=3, visits=1, reward=0, ratio=0.0000)
│   │   │   │   └── 12: (action=1, visits=1, reward=0, ratio=0.0000)
│   │   │   ├── 4: (action=0, visits=5, reward=0, ratio=0.0000)
│   │   │   │   ├── 0: (action=3, visits=1, reward=0, ratio=0.0000)
│   │   │   │   ├── 8: (action=1, visits=1, reward=0, ratio=0.0000)
│   │   │   │   ├── 4: (action=0, visits=1, reward=0, ratio=0.0000)
│   │   │   │ 